In [ ]:
import os

os.environ[
    "OMP_NUM_THREADS"
] = "64"  # for jupyter.nersc.gov otherwise the notebook only uses 2 cores

In [ ]:
from pysm3 import Sky
from pysm3 import units as u

import healpy as hp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import logging
log = logging.getLogger("pysm3")
log.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
log.addHandler(handler)

In [ ]:
models = []
for model_root in ["ksz", "tsz"]:
    models += [model_root + str(i) for i in range(1, 3+1)]

skys = {model:Sky(nside=512, preset_strings=[model]) for model in models}

freq = 100 * u.GHz

m = {model:skys[model].get_emission(freq) for model in models}

In [ ]:
for model in models:
    hp.projview(m[model][0].value, unit=m[model][0].unit, min=-100, max=0)
    plt.title(model)